In [ ]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql.types import *
import pandas as pd
from hdfs import InsecureClient
import os
import re
from hdfs.ext.kerberos import KerberosClient 

In [ ]:
spark  = SparkSession.builder.master("local[1]").appName("importSequence1")\
    .config("spark.driver.memory","4g")\
    .config("spark.executor.memory","4g")\
    .config("spark.driver.maxResultSize","2g")\
    .getOrCreate()  

In [ ]:
rdd=spark.sparkContext.textFile("/Users/brzoskpm/blast_databases/hg38.fa")

In [ ]:
sequence="".join(rdd.collect()[1:len(rdd.collect())])

In [ ]:
#instead of creating list out of rdd, just use collect in the lamda so that I don;t have to convert back into rdd 
#using parrlelize which mamkes it parralelizerdd isntead of mappartition rdd

In [ ]:
#build kmers
def build_kmers(sequence, ksize):
    kmers = pd.DataFrame()
    kmerList = []
    indexList=[]
    n_kmers = len(sequence) - ksize + 1
    #f = open("/Users/karlbrzoska/Documents/BigDataManagementProject/kmers.txt", "w")
    for i in range(n_kmers):
        kmerList.append(sequence[i:i + ksize])
        indexList.append(i)

  
     
    #kmers['index']=indexList
    kmers['kmers']=kmerList
    
        
    return kmers

In [ ]:
def create_kmersrdd(rdd):
    list_of_rdd_kmers =[]
    sequence_list= rdd.collect()
    for i in sequence_list:
        list_of_rdd_kmers.append(spark.sparkContext.parallelize(build_kmers(i, 100)))
    return list_of_rdd_kmers

In [ ]:
kmers=build_kmers(sequence, 30)

In [ ]:
class HDFSPublisher:

  # df is the datasource we wish to write to HDFS
  # filename is what we want the name of the file to be on HDFS
  # this function will pass through to the HDFS client write()

  def write(self, df: pd.DataFrame, filename: str) -> object:
    
    host = 'http://localhost:9870'
    my_client = KerberosClient(host)

    # remember we received the DataFrame and the filename as parameters
    with my_client.write(filename, encoding='utf-8') as writer:
      df.to_csv(writer, header=False)

In [ ]:
publisher = HDFSPublisher()
publisher.write(kmers, 'kmersTest.csv')

In [ ]:
df = spark.read.csv('hdfs://localhost:9000/user/dr.who/kmers.csv')

In [ ]:
df.groupBy('_c1').count().write.csv('hdfs://localhost:9000/user/dr.who/kmersCount.csv')

In [ ]:
client = InsecureClient('http://localhost:9870/')

In [ ]:
client.list('/user/dr.who')

In [ ]:
#split file
with open("/Users/brzoskpm/Karl/genomes/hg38.fa",'r') as data_file:
    out = open("dummy", "w")
    for line in data_file:
        x = re.search("^>", line)
        if (x):
            out.close()
            #try: out.close()
            tmp=line.split()
            name = tmp[0].replace(">","")
            out = open(name, "w")
            out.write(line)
            print(line)
        else:
            out.write(line)
            print(line)
        
out.close()
        
        #data = line.split('>')
        #print(line)

In [ ]:
i=1
import subprocess
import os
for filename in os.listdir("/Users/brzoskpm/Karl/genomes/"):
    print(filename)
    rdd=spark.sparkContext.textFile("file:///Users/brzoskpm/Karl/genomes/"+filename)
    print("rdd finished")
    sequence="".join(rdd.collect()[1:len(rdd.collect())])
    print("sequence finished")
    #list=[]
    #list.append(sequence)
    kmers=build_kmers(sequence, 30)
    print("kmers built")
    file_to_hadoop="/Users/brzoskpm/Karl/genomes/kmer"+filename
    kmers.to_csv(file_to_hadoop, index=False)
    print("write kmers to local file",file_to_hadoop )
    #publisher.write(kmers, f'kmersChr{i}.csv')
    subprocess.call(f'hdfs dfs -copyFromLocal {file_to_hadoop}',shell=True)
    print("written to hdfs")
    i=i+1
    

In [ ]:
kmers.to_csv("/Users/brzoskpm/Karl/genomes/test", index=False)

In [ ]:
#spark1 = SparkSession.builder.master("local").appName("hdfs_test").getOrCreate()

In [ ]:
data=spark.read.csv("hdfs://localhost:9000/user/brzoskpm/kmerchr22")

In [ ]:
kmerCount = data.groupBy("_c0").count().show()

In [ ]:
#THIS IS WHERE MONGODB HAPPENS

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
appName = "PySpark MongoDB Examples"
master = "local"
# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/BigDataManagementProject.Kmers") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/BigDataManagementProject.Kmers") \
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.2')\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.3.0/libexec/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/karlbrzoska/.ivy2/cache
The jars for the packages stored in: /Users/karlbrzoska/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19662785-ca1b-4296-8eb8-34df8597cacb;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;2.4.2 in central
	found org.mongodb#mongo-java-driver;3.12.5 in central
:: resolution report :: resolve 167ms :: artifacts dl 6ms
	:: modules in use:
	org.mongodb#mongo-java-driver;3.12.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;2.4.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||  

22/12/13 23:55:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
T = spark.read.csv('/Users/karlbrzoska/Documents/genomes/kmerchrY.count.csv').toDF('Kmer', 'Count')

In [3]:
T.write.format("mongo").option('uri', 'mongodb://127.0.0.1')\
    .option('database', 'BigDataManagementProject') \
    .option('collection', 'ChrY') \
    .mode("overwrite") \
    .save()

In [ ]:
kmerCount.write.format("mongo").option('uri', 'mongodb://127.0.0.1')\
    .option('database', 'BigDataManagementProject') \
    .option('collection', 'ChrY') \
    .mode("overwrite") \
    .save()